# core

In [ ]:
#| default_exp core

In [ ]:
#| export
from fastcore.all import *
from yt_dlp import YoutubeDL
import json, datetime, httpx

In [ ]:
#| hide
from nbdev.showdoc import *

## Videos

In [ ]:
video_url = 'https://www.youtube.com/watch?v=8SF_h3xF3cE'

In [ ]:
#| export
class YTVideo:
    def __init__(self, data:dict):
        store_attr()
        self.subtitles,self.chapters = None,None
    
    def __repr__(self):
        flds = ['webpage_url','title','language','uploader','categories','tags']
        flds = [f'{o}={self.data[o]!r}' for o in flds if o in self.data]
        if 'duration' in self.data: flds.append(f'duration={datetime.timedelta(seconds=self.data["duration"])}')
        for o in ['automatic_captions','subtitles','chapters','heatmap']: flds.append(f'have_{o}={o in self.data}')
        sig = ', '.join(flds)
        return f'YTVideo({sig})'

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTVideo, url:str, quiet:bool=True)->YTVideo:
    opts = {
        'writedescription':True, 'writesubtitles':True, 'writeautomaticsub':True, 'no_warnings':True, 'skip_download':True,
        'subtitlesformat':'srt', 'quiet':quiet,
    }
    with YoutubeDL(opts) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTVideo(data)

In [ ]:
video = YTVideo.from_url(video_url)
video

YTVideo(webpage_url='https://www.youtube.com/watch?v=8SF_h3xF3cE', title='Practical Deep Learning for Coders: Lesson 1', language='en', uploader='Jeremy Howard', categories=['Education'], tags=['deep learning', 'fastai'], duration=1:22:55, have_automatic_captions=True, have_subtitles=True, have_chapters=True, have_heatmap=True)

### Subtitles

In [ ]:
#| export
@patch
def subtitles_url(self:YTVideo, language:str=None):
    lang = ifnone(language, self.data.get('language'))
    d = self.data.get('subtitles', self.data.get('automatic_captions'))
    if d is None: return
    return next(o['url'] for o in d[lang] if o['ext']=='srt')

In [ ]:
subtitle_url = video.subtitles_url()
subtitle_url

'https://www.youtube.com/api/timedtext?v=8SF_h3xF3cE&ei=jtJSaZjeLKX1ir4Pzc7AmAo&caps=asr&opi=112496729&xoaf=5&xowf=1&hl=en&ip=0.0.0.0&ipbits=0&expire=1767060734&sparams=ip%2Cipbits%2Cexpire%2Cv%2Cei%2Ccaps%2Copi%2Cxoaf&signature=1684FC31CDE9F30CA891131570F6E4E07FA4B976.6177D2A4151D083FBC379B2D5BED4D1FA67E2873&key=yt8&lang=en&fmt=srt'

In [ ]:
subs = httpx.get(subtitle_url).text

In [ ]:
block = subs.split('\n\n')[0]
block

'1\n00:00:02,000 --> 00:00:10,000\nWelcome to Practical Deep Learning for coders,\xa0\nlesson one. This is version five of this course,\xa0\xa0'

In [ ]:
#| export
_subtitle_entry_pat = re.compile(r'(\d+)\n(\d+:\d+:\d+),\d+ --> (\d+:\d+:\d+),\d+\n(.+)', re.DOTALL)

class SubtitleEntry:
    __repr__ = basic_repr()
    def __init__(self, index:int, start:datetime.timedelta, end:datetime.timedelta, text:str): store_attr()

    @classmethod
    def from_str(cls, s:str)->'Self':
        if s.count('\n')<2: return None
        match = _subtitle_entry_pat.match(s.strip())
        text = re.sub(r'[\n\xa0]+', ' ', match.group(4))
        return cls(int(match.group(1)), match.group(2), match.group(3), text)

In [ ]:
SubtitleEntry.from_str(block)

SubtitleEntry(index=1, start='00:00:02', end='00:00:10', text='Welcome to Practical Deep Learning for coders, lesson one. This is version five of this course,')

In [ ]:
#| export
class Subtitles:
    def __init__(self, entries:L): store_attr()

    def __repr__(self):
        sig = ', '.join([f'entries={len(self.entries)}', f'last_ts={self.entries[-1].end!r}'])
        return f'Subtitles({sig})'

    @classmethod
    def from_str(cls, s:str)->'Self': 
        entries = L.split(s.strip(), '\n\n').map(SubtitleEntry.from_str).filter()
        return cls(entries)

In [ ]:
Subtitles.from_str(subs)

Subtitles(entries=769, last_ts='01:22:55')

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:Subtitles, url:str)->Subtitles: return cls.from_str(httpx.get(url).text)

In [ ]:
subtitles = Subtitles.from_url(subtitle_url)
subtitles

Subtitles(entries=769, last_ts='01:22:55')

In [ ]:
#| export
@patch
def format_subs(self:Subtitles)->str:
    "Formats subtitles to use in LLMs."
    return '\n'.join(self.entries.map(lambda o: f'[{o.start}] {o.text}'))

In [ ]:
subtitles.format_subs()[:100]

'[00:00:02] Welcome to Practical Deep Learning for coders, lesson one. This is version five of this c'

In [ ]:
#| export
@patch
@delegates(YTVideo.subtitles_url)
def fetch_subtitles(self:YTVideo, force:bool=False, **kwargs)->YTVideo:
    if force or (self.subtitles is None): self.subtitles = Subtitles.from_url(self.subtitles_url(**kwargs))
    return self

In [ ]:
video.fetch_subtitles()
video.subtitles

Subtitles(entries=769, last_ts='01:22:55')

In [ ]:
#| export
@patch
@delegates(YTVideo.fetch_subtitles)
def format_subs(self:YTVideo, **kwargs)->str:
    self.fetch_subtitles(**kwargs)
    if self.subtitles is None: return
    return self.subtitles.format_subs()

In [ ]:
video.format_subs()[:100]

'[00:00:02] Welcome to Practical Deep Learning for coders, lesson one. This is version five of this c'

### Chapters

In [ ]:
#| export
def _format_chapter(s:dict)->str:
    start = datetime.timedelta(seconds=s['start_time'])
    return f'[{start}] {s["title"]}'

In [ ]:
_format_chapter(video.data['chapters'][0])

'[0:00:00] Introduction'

In [ ]:
#| export
@patch
def format_chapters(self:YTVideo)->str:
    if 'chapters' not in self.data: return
    return '\n'.join(map(_format_chapter, self.data['chapters']))

In [ ]:
print(video.format_chapters()[:120])

[0:00:00] Introduction
[0:00:25] What has changed since 2015
[0:01:20] Is it a bird
[0:02:09] Images are made of numbers


### Summary prompt

In [ ]:
#| export
@patch
@delegates(YTVideo.fetch_subtitles)
def create_summary_prompt(self:YTVideo, **kwargs)->str:
    self.fetch_subtitles(**kwargs)
    transcript = self.format_subs()
    if transcript is None: return
    description = self.data.get('description')
    chapters = self.format_chapters()
    prompt = f'''In the <transcript> tag we have transcript of a video with the title: {self.data['title']!r}. \
Analyse the transcript to generate a detailed summary of the content of the video.

Your task:
<task>
1. Start with a 2-3 sentence overview of the entire video
2. Identify where major topics begin and end (chapters)
3. Summarise each chapter's key concepts, it should have very high information value. Do not miss any important topic.
4. Extract all resources mentioned, e.g. links, books, papers, videos, YouTube channels etc.
</task>

Format your response in Markdown:
<format>
- Start with "## Overview" followed by the 2-3 sentence summary
- For each chapter use: "## [Chapter Title] - HH:MM:SS", e.g.: "## Introduction - 00:02:30"
- End with a "## Resources" section containing a list of the resources mentioned through the video, including a brief context.
<format>

Here is the transcript with timestamps:
<transcript>
{transcript}
</transcript>'''

    if description is not None:
        prompt += f'''

Here is the video description:
<video-description>
{description}
</video-description>
'''

    if chapters is not None:
        prompt += f'''

Incase it is helpful, here are the chapters defined in the video. However, please use timestamps from the transcript when possible when constructing timestamped links. 
<video-chapters>
{chapters}
</video-chapters>
'''

    prompt += '''

Keep the following writing guidelines in mind:
<guidelines>
1. Do not add filler words. 
2. Make every sentence information-dense without repetition.
3. Get to the point while providing necessary context.
4. Use short words and fewer words.
5. Avoid overusing bullet points. Prefer flowing prose that combines related concepts. Use lists only for truly distinct items.
</guidelines>

Please go ahead and write the summary.'''
    return prompt

In [ ]:
prompt = video.create_summary_prompt()
transcript_a,transcript_b = prompt.index('\n<transcript>'),prompt.index('\n</transcript>')
description_b = prompt.index('\n</video-description>')
chapters_b = prompt.index('\n</video-chapters>')
print(
    prompt[:transcript_a+200], '\n...',
    prompt[transcript_b:transcript_b+120], '...',
    prompt[description_b:description_b+270], '\n...',
    prompt[chapters_b:],
)

In the <transcript> tag we have transcript of a video with the title: 'Practical Deep Learning for Coders: Lesson 1'. Analyse the transcript to generate a detailed summary of the content of the video.

Your task:
<task>
1. Start with a 2-3 sentence overview of the entire video
2. Identify where major topics begin and end (chapters)
3. Summarise each chapter's key concepts, it should have very high information value. Do not miss any important topic.
4. Extract all resources mentioned, e.g. links, books, papers, videos, YouTube channels etc.
</task>

Format your response in Markdown:
<format>
- Start with "## Overview" followed by the 2-3 sentence summary
- For each chapter use: "## [Chapter Title] - HH:MM:SS", e.g.: "## Introduction - 00:02:30"
- End with a "## Resources" section containing a list of the resources mentioned through the video, including a brief context.
<format>

Here is the transcript with timestamps:
<transcript>
[00:00:02] Welcome to Practical Deep Learning for coders

## Playlists

In [ ]:
playlist_url = 'https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU'

In [ ]:
#| export
class YTPlaylist:
    def __init__(self, data:dict): store_attr()
    def __repr__(self):
        flds = ['webpage_url', 'title', 'channel', 'playlist_count']
        sig = ', '.join(f'{o}={self.data[o]!r}' for o in flds)
        return f'YTPlaylist({sig})'

In [ ]:
#| export
@patch(cls_method=True)
def from_url(cls:YTPlaylist, url:str, quiet:bool=True)->YTPlaylist:
    with YoutubeDL({'flat_playlist':True, 'extract_flat':True, 'quiet':quiet}) as ydl:
        data = ydl.extract_info(url, download=False)
        return YTPlaylist(data)

In [ ]:
playlist = YTPlaylist.from_url(playlist_url)
playlist

YTPlaylist(webpage_url='https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU', title='Practical Deep Learning for Coders', channel='Jeremy Howard', playlist_count=8)

In [ ]:
playlist.data['description']

'This free course is designed for people with some coding experience who want to learn how to apply deep learning and machine learning to practical problems.\n\nThis course covers topics such as how to:\n- Build and train deep learning models for computer vision, natural language processing, tabular analysis, and collaborative filtering problems\n- Create random forests and regression models\n- Deploy models\n- Use PyTorch, the world’s fastest growing deep learning software, plus popular libraries like fastai and Hugging Face\n\nThere are 9 lessons, and each lesson is around 90 minutes long. The course is based on our 5-star rated book, which is freely available online.\n\nYou don’t need any special hardware or software — we’ll show you how to use free resources for both building and deploying models. You don’t need any university math either — we’ll teach you the calculus and linear algebra you need during the course.'

In [ ]:
#| export
@patch
@delegates(YTVideo.from_url)
def get_videos(self:YTPlaylist, **kwargs)->L:
    return parallel()
    return L(self.data['entries'][:2]).map(lambda o: YTVideo.from_url(o['url'], **kwargs))

playlist.get_videos()

[YTVideo(webpage_url='https://www.youtube.com/watch?v=8SF_h3xF3cE', title='Practical Deep Learning for Coders: Lesson 1', language='en', uploader='Jeremy Howard', categories=['Education'], tags=['deep learning', 'fastai'], duration=1:22:55, have_automatic_captions=True, have_subtitles=True, have_chapters=True, have_heatmap=True), YTVideo(webpage_url='https://www.youtube.com/watch?v=F4tvM4Vb3A0', title='Lesson 2: Practical Deep Learning for Coders 2022', language='en', uploader='Jeremy Howard', categories=['Education'], tags=['deep learning', 'fastai'], duration=1:16:42, have_automatic_captions=True, have_subtitles=True, have_chapters=True, have_heatmap=True)]

In [ ]:
res = parallel(lambda o: o**2, range(2,7), threadpool=True, n_workers=4)

In [ ]:
show_doc(parallel)

KeyboardInterrupt: 

test_eq(parallel(_add_one, inp, threadpool=True, n_workers=2), exp)

# -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()